## Chapter 3: Linear and Logistic Regression

This follows the exercises in the book *Tensorflow for Deep Learning*.

Sources:

- [1] *Tensorflow for Deep Learning*

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# Generate synthetic  regression data
# Fake linear model y = wx + b + N
N = 100
w_true = 5
b_true = 2
noise_scale = 0.1
x_np = np.random.rand(N, 1)
noise = np.random.normal(scale=noise_scale, size=(N,1))
y_np = np.reshape(w_true*x_np + b_true + noise, (-1))

In [ ]:
N = 100
c = N/2

In [ ]:
# Generate synthetic classification data
# Fake classification of "ones" (-1, -1) + N and "zeros" (1, 1) + N
N = 100
half = (int)(N/2)
# Zeros are a gausian centered at (-1, -1)
x_zeros = np.random.multivariate_normal(
    mean=np.array((-1, -1)), cov=.1*np.eye(2), size=(half,))
y_zeros = np.zeros((half,))
# Ones are a gausian centered at (1, 1)
x_ones = np.random.multivariate_normal(
    mean=np.array((1, 1)), cov=.1*np.eye(2), size=(half,))
y_ones = np.ones((half,))
# Concatenate data together
x_np = np.vstack([x_zeros, x_ones])
y_np = np.concatenate([y_zeros, y_ones])

In [ ]:
# Tensorflow placeholders are used for input data into a tf graph
tf.placeholder(tf.float32, shape=(2,2))

In [ ]:
# Use feed dictionary to feed values into a graph in a session
a = tf.placeholder(tf.float32, shape=(1,))
b = tf.placeholder(tf.float32, shape=(1,))
c = a + b
with tf.Session() as sess:
    c_eval = sess.run(c, {a: [1.], b: [2.]})
    print(c_eval)

In [ ]:
# Use name scopes to help with Tensorboard Viz, debug, etc
N = 5
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (N, 1))
    y = tf.placeholder(tf.float32, (N,))

In [ ]:
# Put your train op inside its own name scope
learning_rate = 0.001
with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
# Use tf.gradients to get the gradients directly
# useful for debugging
W = tf.Variable((3,))
l = tf.reduce_sum(W)
gradW = tf.gradients(l, W)
gradW

In [ ]:
# Use summary writers to log things
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()
    
train_writer = tf.summary.FileWriter('/tmp/lr-train', tf.get_default_graph())

In [ ]:
n_steps = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Train the model
    for i in range(n_steps):
        feed_dict = {x: x_np, y: y_np}
        
        _, summary, loss = sess.run([train_op, merged, l], feed_dict=feed_dict)
        print("step %d, loss: %f" % (i, loss))
        train_writer.add_summary(summary, i)

### Linear Regresssion

In [ ]:
# Create the graph
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (N, 1))
    y = tf.placeholder(tf.float32, (N,))
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal((1,1)))
    b = tf.Variable(tf.random_normal((1,)))
with tf.name_scope("prediction"):
    y_pred = tf.matmul(x, W) + b
with tf.name_scope("loss"):
    loss = tf.reduce_sum((y - y_pred) ** 2)
with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", loss)
    merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter('/tmp/lr-train', tf.get_default_graph())

# Create the session, feed the data in, and train the model
n_steps = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Train model
    for i in range(n_steps):
        feed_dict = {x: x_np, y: y_np}
        _, summary, l = sess.run([train_op, merged, loss],
                                   feed_dict=feed_dict)
        print("step %d, loss: %f" % (i, l))
        train_writer.add_summary(summary, i)